In [1]:
import time
import math
import random
import itertools as its
import numpy as np
import pandas as pd
import tqdm

from collections import Counter

In [2]:
dt = pd.read_csv("../data/user2up.csv")
dt.rename(columns={"mid":"up","from":"user"},inplace=True)
dt.up = dt.up.astype(int)
dt_fmt = dt[["user","up"]]

In [3]:
dt_fmt

,user,up
0,81922,326499679
1,81922,316568752
2,81922,9617619
3,81922,213741
4,81922,676498418
...,...,...
697970,519570206,456664753
697971,519570206,14114387
697972,519570206,504739753
697973,519570206,414021604


In [4]:
class UserCF():
    def __init__(self,data,K,N,split_n=0.7,ignore_user_n=10,ignore_up_n=10,seed=123):
        self.data = data
        self.K = K
        self.N = N
        self.split_n = split_n
        self.ignore_user_n = ignore_user_n
        self.ignore_up_n = ignore_up_n
        self.seed = seed
        self.data_format(data)

        
    def data_format(self,data):
        

        # 为了保证在切分用户行为数据的时候,训练与测试集均保证一定数量
        # 此处过滤掉行为数量过少的UP
        up_behavior_count = data.groupby("up").count()
        ups = set(up_behavior_count[up_behavior_count.user>=self.ignore_up_n].index)
        
        user_behavior_count = data.groupby("user").count()
        users = set(user_behavior_count[user_behavior_count.up>=self.ignore_user_n].index)
        
        # 此处过滤掉行为数量过少的User和UP
        data = data[(data.up.isin(ups)&(data.user.isin(users)))]
        
        # 此处根据过滤后的结果重新获取集合
        users = set(data.user.unique())
        ups = set(data.up.unique())

        data = data.sample(frac=1,random_state=self.seed)

#         # 全随机切分
#         split = int(len(data)*self.split_n)
#         train_df = data[:split]
#         test_df = data[split:]
#         users = train_df.user.unique()
#         ups = test_df.up.unique()
        
        # 基于单用户内的关注行为切分
        train_df = pd.DataFrame()
        test_df = pd.DataFrame()
        for user_tmp in tqdm.tqdm(data.user.unique(),"切割数据"):
            data_user = data[data.user==user_tmp]
            data_user_split = int(len(data_user)*self.split_n)
            train_df = train_df.append(data_user[:data_user_split])
            test_df = test_df.append(data_user[data_user_split:])
        
        
#         # User对UP主的关注集
#         # 切分为训练测试集
#         matrix_user2up = train_df.groupby("user").agg(set)
#         matrix_user2up["train"] = matrix_user2up["up"].apply(self.sample,args=(self.split_n,self.seed))
#         matrix_user2up[["train","test"]] = matrix_user2up["train"].apply(pd.Series)
        # User对UP主的关注集
        # 切分为训练测试集
        matrix_user2up = data.groupby("user").agg(set)
        matrix_user2up["train"] = train_df.groupby("user").agg(set)
        matrix_user2up["test"] = test_df.groupby("user").agg(set)

        
        # UP主对User倒排表
        matrix_up2user = train_df.groupby("up").agg(set)
        matrix_up2user = matrix_up2user.user.to_dict()
        
        train = matrix_user2up.train.to_dict()
        test = matrix_user2up.test.to_dict()
        

        for user in train:
            if train[user] is np.nan:
                train[user] = set()
        
        for user in test:
            if test[user] is np.nan:
                test[user] = set()
        
        # 倒排表matrix_up2user降低计算量
        matrix_user2user = {}
        count = 0
        
        # 遍历倒排表,对存在关注同一个up主的user进行相似度计算
        # TODO 此处考虑使用numpy来做一遍或许更有利于避免重复计算
        for up,sub_users in tqdm.tqdm(matrix_up2user.items(),"用户间相似度"):
            for user_a,user_b in its.combinations(sub_users,2):
                # 排序限定为左小右大
                user_a,user_b = (user_a,user_b) if user_a < user_b else (user_b,user_a)
                # 若无历史结果,则计算
                if matrix_user2user.get((user_a,user_b)) == None:
                    # TODO,应该支持多类相似度算法
                    # 此处为快速实现使用了杰卡德相似度
                    matrix_user2user[(user_a,user_b)] = len(train[user_a]&train[user_b])/len(train[user_a]|train[user_b])
                count += 1
        print("倒排表约使计算量减少至原来的%s倍."%round(count/(len(users)**2),2))
        
        self.data = data
        self.users = users
        self.ups = ups
        self.matrix_user2up = matrix_user2up
        self.matrix_up2user = matrix_up2user
        self.matrix_user2user = matrix_user2user
        self.train = train
        self.test = test
        
#     def sample(self,up_set,train_k,seed):
#         random.seed(seed)
#         train = set(random.sample(up_set,int(len(up_set)*train_k)))
#         test = up_set - train
#         return train,test
    
    def GetRecommendation(self,user):
        user2user = {}
        
        # 计算目标user与其他user的相似度
        for user_other in self.users:
            if user_other == user:
                continue
            # 标准化key
            user_a,user_b = (user,user_other) if user < user_other else (user_other,user)
            user2user[user_other] = self.matrix_user2user.get((user_a,user_b),0)
        
        # TODO 此处的推荐过程可以增加缓存,这样子在仅变动K,N的时候,不用重复计算topK_user和topN_up
        # 根据user2user的value相似度降序排后取前K位
        topK_user = sorted(user2user.items(),key=lambda x: x[1],reverse=True)[:self.K]
        topK_user_ups_list = []
        
        # 取topK_user的关注列表
        for user_other,score in topK_user:
            topK_user_ups_list += self.train[user_other]
        # 注意去掉当前已推荐的
        counter = Counter(topK_user_ups_list)
        counter = {up:n for up,n in counter.items() if up not in self.train[user]}
        topN_up = sorted(counter.items(),key=lambda x: x[1],reverse=True)[:self.N]
        return set(topN_up)

In [5]:
class EvaluationMetrics():
    def __init__(self,train,test,get_rec):
        """
        :param train: {123:{1,2,3,4,5}}
        :param test: {123:{1,2,3,4,5}}
        :param get_rec: recs = get_rec(user)
        """
        self.train = train
        self.test = test
        self.get_rec = get_rec
        self.recs = self.get_rec_all_user()
        self.hit = self._hit()
    
    # 获得所有test用户的推荐结果
    def get_rec_all_user(self):
        # recs = {1:{(64554310,9),(123142,8)}}
        recs = {}
        for user in tqdm.tqdm(self.train,"推荐中"):
            recs[user] = self.get_rec(user)
        return recs
    
    # 分子/命中
    def _hit(self):
        hit = 0
        for user in self.test:
            sub_test = self.test[user]
            rec = {item for item,score in self.recs[user]}
            hit += len(sub_test&rec)
        return hit
    
    # 精确率
    def precision(self):
        count = 0
        for user in self.test:
            count += len(self.recs[user])
        return round(self.hit / count * 100,2)
    
    # 召回率
    def recall(self):
        count = 0
        for user in self.test:
            count += len(self.test[user])
        return round(self.hit / count * 100,2)
    
    # 覆盖率
    def coverage(self):
        all_items = set()
        rec_items = set()
        for user,items in self.train.items():
            all_items = all_items|items
        # TODO 这边考虑直接以模型的data.ups.unique()去处理避免重复计算
        for user,items in self.recs.items():
            rec_items = rec_items|set([item for item,score in items])
        return round(len(rec_items) / len(all_items) * 100,2)
    
    # 新颖度
    def popularity(self):
        # 计算物品的流行度
        item_pop = {}
        for user in self.train:
            for item in self.train[user]:
                if item not in item_pop:
                    item_pop[item] = 0
                item_pop[item] += 1

        num, pop = 0, 0
        for user in self.test:
            rank = self.recs[user]
            for item, score in rank:
                # 取对数
                pop += math.log(1 + item_pop[item])
                num += 1
        return round(pop / num, 6)
    
    def evaluate(self):
        evaluation_metrics = {'precision': self.precision(),
                              'recall': self.recall(),
                              'coverage': self.coverage(),
                              'popularity': self.popularity()}
        print('EvaluationMetrics:', evaluation_metrics)
        return evaluation_metrics

In [6]:
class Experiment():
    def __init__(self,model,T):
        self.model = model
        self.T = T
    
    def validate(self):
        resulst = EvaluationMetrics(self.model.train,self.model.test,self.model.GetRecommendation)
        return resulst.evaluate()

In [7]:
results = pd.DataFrame(columns=["model","split_n","K","N",
                                "ignore_user_n","ignore_up_n","seed",
                                "precision","recall","coverage","popularity"])

In [ ]:
for ignore_user_n,ignore_up_n,seed in its.product([30,10],[100,70,40,15],[1,2,3]):
    model_usercf = UserCF(dt_fmt,K=30,N=30,split_n=0.7,
                          ignore_user_n=ignore_user_n,ignore_up_n=ignore_up_n,seed=seed)
    for K,N in its.product([10,30,50,80,120],[20,50,90]):
        print(K,N)
        time.sleep(0.5)
        model_usercf.K = K
        model_usercf.N = N
        usercf_exp = Experiment(model_usercf,5)

        res = usercf_exp.validate()
        res["model"] = "UserCF"
        res["split_n"] = 0.7
        res["ignore_user_n"] = ignore_user_n
        res["ignore_up_n"] = ignore_up_n
        res["seed"] = seed
        res["K"] = K
        res["N"] = N
        results = results.append(res,ignore_index=True)

用户间相似度: 100%|██████████████████████████████████████████████████████████████████| 619/619 [00:21<00:00, 29.45it/s]


倒排表约使计算量减少至原来的0.23倍.
10 20


推荐中: 100%|█████████████████████████████████████████████████████████████████████| 7181/7181 [01:03<00:00, 112.97it/s]


EvaluationMetrics: {'precision': 5.2, 'recall': 16.18, 'coverage': 100.0, 'popularity': 5.321696}
10 50


推荐中: 100%|█████████████████████████████████████████████████████████████████████| 7181/7181 [01:03<00:00, 113.08it/s]


...

切割数据:   0%|                                                                               | 0/8563 [00:00<?, ?it/s]

EvaluationMetrics: {'precision': 3.38, 'recall': 35.47, 'coverage': 99.81, 'popularity': 5.319271}


用户间相似度: 100%|████████████████████████████████████████████████████████████████| 2126/2126 [00:32<00:00, 65.05it/s]


倒排表约使计算量减少至原来的0.2倍.
10 20


推荐中:  12%|████████▌                                                             | 1046/8563 [00:11<01:25, 87.47it/s]

In [ ]:
for ignore_user_n,ignore_up_n,seed in its.product([30,10],[100,70,40,15],[1,2,3]):
    model_usercf = UserCF(dt_fmt,K=30,N=30,split_n=0.7,
                          ignore_user_n=ignore_user_n,ignore_up_n=ignore_up_n,seed=seed)
    for K,N in its.product([170,330,500,1000],[20,50,90]):
        print(K,N)
        time.sleep(0.5)
        model_usercf.K = K
        model_usercf.N = N
        usercf_exp = Experiment(model_usercf,5)

        res = usercf_exp.validate()
        res["model"] = "UserCF"
        res["split_n"] = 0.7
        res["ignore_user_n"] = ignore_user_n
        res["ignore_up_n"] = ignore_up_n
        res["seed"] = seed
        res["K"] = K
        res["N"] = N
        results = results.append(res,ignore_index=True)

In [ ]:
len(model_usercf.train)

In [ ]:
model_usercf.train[384565347]

In [ ]:
model_usercf.matrix_user2up.loc[384565347,:]

In [ ]:
dt_fmt[dt_fmt.user==384565347]

In [ ]:
results